In [1]:
from matplotlib import pyplot as plt
from sklearn.discriminant_analysis import StandardScaler
from sklearn.metrics import silhouette_score
import streamlit as st
import pandas as pd 
import numpy as np 
from nba_api.stats.endpoints import LeagueGameLog
from nba_api.stats.endpoints import teamgamelogs
from sklearn.cluster import KMeans
import datetime
import os

In [5]:

log = teamgamelogs.TeamGameLogs(season_nullable="2024-25", last_n_games_nullable=25)
df = log.get_data_frames()[0]
    

df = df[['TEAM_NAME','MIN', 'FGA', 'FG_PCT', 'FG3A', 'FG3_PCT', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA',
       'PF', 'PFD', 'PTS', 'PLUS_MINUS']]
df_agg = df.groupby('TEAM_NAME').mean().reset_index()
### returns teams data aggregated df 

scaler = StandardScaler()
df = scaler.fit_transform(df_agg.drop(columns=["TEAM_NAME"]))
df = pd.DataFrame(df,columns = df_agg.columns[1:])
df['TEAM_NAME'] = df_agg['TEAM_NAME']

df
    

,MIN,FGA,FG_PCT,FG3A,FG3_PCT,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,TEAM_NAME
0,-0.363422,0.654784,0.387172,-0.011897,0.197953,0.465930,-0.987296,-0.129260,-0.388295,-0.382445,1.000866,0.894961,0.997243,1.439144,-0.528803,0.568004,0.183888,0.633011,-0.271526,Atlanta Hawks
1,-1.142183,-0.774149,-0.050650,2.578632,1.427754,-2.225337,1.537899,-0.344694,0.427125,0.098581,-0.279238,-1.922299,-1.427427,1.014478,-2.682539,-1.415353,-1.741380,-0.114934,1.072369,Boston Celtics
2,-1.142183,-1.084038,-1.799958,0.241399,-2.074607,-0.723234,-0.362802,1.351848,-2.182219,-0.774392,-0.881640,-0.021497,0.879921,1.651476,1.047101,1.266369,-0.556600,-2.037155,-0.588482,Brooklyn Nets
3,-1.142183,0.052222,-2.324949,-0.506976,-2.235931,-1.662048,0.896772,0.543971,-0.178616,0.223291,-0.919291,0.657361,-0.997243,-0.754961,0.836980,-0.716988,-1.445185,-2.306415,-2.154246,Charlotte Hornets
4,0.415339,0.981890,-0.773750,1.484853,-0.496441,-0.681509,1.020764,-0.667845,1.289140,0.543975,0.398464,0.046389,-0.684383,-0.188740,0.889510,-0.633184,-1.148990,0.072052,-0.499734,Chicago Bulls
5,-0.363422,0.809729,0.739807,1.358205,1.098092,0.883180,0.208770,0.678617,1.359033,1.488211,0.153738,-0.394869,-0.136877,-1.391959,-0.791454,0.204854,0.406034,1.605341,1.940829,Cleveland Cavaliers
6,-0.363422,-0.274883,0.210855,-0.599084,0.756741,-0.326846,-0.402116,-1.206430,-0.784356,-1.397944,-0.580439,-0.904012,-0.997243,0.731368,0.259149,-1.499157,-0.741722,-0.174770,-1.076595,Dallas Mavericks
7,-0.363422,0.499840,2.122613,-1.151730,1.121472,-1.119622,0.387197,0.193891,0.520316,0.526159,2.130370,0.182160,-0.371522,-0.188740,0.259149,-0.577315,-1.223038,1.231368,0.457474,Denver Nuggets
8,-1.142183,0.310463,1.118197,-1.163243,-0.367849,1.383881,-0.488305,0.113103,0.799888,0.686501,0.549064,0.419760,1.583857,0.872923,0.521799,1.825061,1.294619,0.954628,1.237186,Detroit Pistons
9,-1.142183,0.155518,-0.496397,1.519393,-0.091962,1.467331,0.738002,0.732475,-0.318402,0.241107,1.735043,-0.496698,1.544749,-0.047185,-0.371213,0.847350,1.220570,0.543258,1.230847,Golden State Warriors
